In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 44.8 MB/s eta 0:00:00


In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision.models import DenseNet201_Weights, VGG19_Weights
import json


# YOLOv5 모델 로드
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/지우님/yolo_연결/team2_yolov5m_100.pt')
yolo_model.imgsz = 416
yolo_model.conf = 0.3
yolo_model.iou = 0.5
yolo_model.agnostic = True
yolo_model.stride = 1
yolo_model.max_det = 1000
yolo_model.line_thickness = 2

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50의 입력 크기에 맞게 조정
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

# 이미지 로드 및 RGB 변환
img_path = '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/지우님/yolo_연결/b.jpeg'
img = cv2.imread(img_path)
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# YOLOv5 모델을 사용하여 탐지
results = yolo_model(image_rgb)

# 자르기
def cropped_img(annotation, resnet_model, results, model_class_id):
    detection_data = results.xyxy[0].cpu().numpy()  # YOLOv5의 탐지 결과 추출
    json_results = []
    # 클래스별로 최대 신뢰도를 추적하기 위한 변수
    max_confidence = -1
    best_box = None
    for *box, conf, cls in detection_data:
        class_id = int(cls)
        if class_id == model_class_id:
            if conf > max_confidence:  # 더 높은 신뢰도일 경우 갱신
                max_confidence = conf
                best_box = box
    if best_box is not None:
        x1, y1, x2, y2 = map(int, best_box)
        # 바운딩 박스 영역을 자르기
        cropped_img = img[y1:y2, x1:x2]
        # OpenCV 이미지를 PIL 이미지로 변환
        cropped_img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
        # 이미지 전처리
        input_tensor = transform(cropped_img_pil).unsqueeze(0).to(device)
        # 모델에 이미지 입력하여 예측
        with torch.no_grad():
            prediction = resnet_model(input_tensor)
            predicted_class = torch.argmax(prediction, dim=1).item()
        # 결과를 JSON 포맷으로 저장
        json_results.append({
            "class": annotation,
            "predicted_class": predicted_class
        })
    return json_results

##########################################################################################
# 클래스 ID와 해당 모델 매핑
model_class_id_map = {
    'pigmentation_forehead': 1,  # 이마
    'pigmentation_cheek_l': 5,   # 왼쪽 볼
    'pigmentation_cheek_r': 6,   # 오른쪽 볼
    'wrinkle_perocular_r' : 4,   # 오른쪽 눈가
    'wrinkle_perocular_l' : 3,    # 왼쪽 눈가
    'wrinkle_forehead': 1,       # 이마
    'wrinkle_glabellus': 2,       # 미간
    'chin_sagging' : 8,          # 턱
    'l_cheek_pore' : 5,          # 왼쪽 볼
    'r_cheek_pore' : 6           # 오른쪽 볼
}

#################################################################################################
# 모델 정의
class DenseNet201_VGG19_Ensemble(nn.Module):
    def __init__(self, num_classes, drop_out):
        super(DenseNet201_VGG19_Ensemble, self).__init__()

        # DenseNet201 정의
        self.densenet = models.densenet201(weights=DenseNet201_Weights.DEFAULT)
        densenet_features = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Identity()  # 최종 분류기를 제거하고 특징만 추출

        # VGG19 정의
        self.vgg = models.vgg19(weights=VGG19_Weights.DEFAULT)
        vgg_features = self.vgg.classifier[0].in_features
        self.vgg.classifier = nn.Identity()  # 최종 분류기를 제거하고 특징만 추출

        # 두 모델의 특징을 결합하는 계층
        self.classifier = nn.Sequential(
            nn.Linear(densenet_features + vgg_features, 1024),
            nn.ReLU(),
            nn.Dropout(p=drop_out),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=drop_out),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # DenseNet201 특징 추출
        densenet_features = self.densenet(x)

        # VGG19 특징 추출
        vgg_features = self.vgg(x)

        # 두 특징을 결합
        combined_features = torch.cat((densenet_features, vgg_features), dim=1)

        # 최종 분류
        output = self.classifier(combined_features)
        return output
###############################################################################
# 모델 설정
model_settings = {
    'pigmentation_forehead': {
        'path': '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/forehead_pigmentation.pth',
        'num_classes': 2,
        'dropout_rate': 0.5
    },
    'pigmentation_cheek_l': {
        'path': '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/cheek_pigmentation.pth',
        'num_classes': 2,
        'dropout_rate': 0.5
    },
    'pigmentation_cheek_r': {
        'path': '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/cheek_pigmentation.pth',
        'num_classes': 2,
        'dropout_rate': 0.5
    },

    'wrinkle_perocular_r': {
        'path':  '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/persocular_wrinkle.pth',
        'num_classes': 2,
        'dropout_rate': 0.3
    },
    'wrinkle_perocular_l': {
        'path':   '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/persocular_wrinkle.pth',
        'num_classes': 2,
        'dropout_rate': 0.3
    },
    'wrinkle_forehead': {
        'path': '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/forehead_wrinkle.pth',
        'num_classes': 2,
        'dropout_rate': 0.5
    },
    'wrinkle_glabellus':{
        'path':  '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/wrinkle_glabellus.pth',
        'num_classes': 2,
        'dropout_rate': 0.5
    },
    'chin_sagging':{
        'path':  '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/chin_sagging.pt',
        'num_classes': 2,
        'dropout_rate': 0.5
    },
    'l_cheek_pore':{
        'path':  '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/cheek_pore.pt',
        'num_classes': 2,
        'dropout_rate': 0.5
    },
    'r_cheek_pore':{
        'path':  '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/final_model/cheek_pore.pt',
        'num_classes': 2,
        'dropout_rate': 0.5
    }
}



# 모델 로드 및 예측
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_results = []
for annotation, class_id in model_class_id_map.items():
    model_settings_for_annotation = model_settings[annotation]
    resnet_model = DenseNet201_VGG19_Ensemble(
        num_classes=model_settings_for_annotation['num_classes'],
        drop_out=model_settings_for_annotation['dropout_rate']

    )
    # state_dict를 로드하고 모델에 적용
    # state_dict의 키 이름을 변경하여 로드
    def rename_state_dict_keys(state_dict, prefix="vgg"):
        renamed_state_dict = {}
        for key, value in state_dict.items():
            if key.startswith("vgg19."):
                new_key = key.replace("vgg19.", f"{prefix}.")
                renamed_state_dict[new_key] = value
            else:
                renamed_state_dict[key] = value
        return renamed_state_dict

# 가중치 로드 및 키 이름 수정
    state_dict = torch.load(model_settings_for_annotation['path'], map_location=device)
    renamed_state_dict = rename_state_dict_keys(state_dict)
    resnet_model.load_state_dict(renamed_state_dict)

    resnet_model.to(device).eval()
    results_for_annotation = cropped_img(annotation, resnet_model, results, class_id)
    all_results.extend(results_for_annotation)
#################################################################################



# JSON 파일로 저장
json_file_path = '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/detection_results.json'
with open(json_file_path, 'w') as f:
    json.dump(all_results, f, indent=2)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-29 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20885262 parameters, 0 gradients, 48.0 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
import json

# JSON 파일을 읽어오는 함수
def load_results_from_json(filename):
    with open(filename, 'r') as json_file:
        results_data = json.load(json_file)
    return results_data

# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/Final_project_2조/02_2. 전처리 및 EDA_이미지/yolo/detection_results.json'


# JSON 파일 읽기
results_data = load_results_from_json(json_file_path)

# 결과 출력
print(json.dumps(results_data, indent=2))

[
  {
    "class": "pigmentation_forehead",
    "predicted_class": 0
  },
  {
    "class": "pigmentation_cheek_l",
    "predicted_class": 1
  },
  {
    "class": "pigmentation_cheek_r",
    "predicted_class": 1
  },
  {
    "class": "wrinkle_perocular_r",
    "predicted_class": 1
  },
  {
    "class": "wrinkle_perocular_l",
    "predicted_class": 1
  },
  {
    "class": "wrinkle_forehead",
    "predicted_class": 1
  },
  {
    "class": "wrinkle_glabellus",
    "predicted_class": 0
  },
  {
    "class": "chin_sagging",
    "predicted_class": 0
  },
  {
    "class": "l_cheek_pore",
    "predicted_class": 1
  },
  {
    "class": "r_cheek_pore",
    "predicted_class": 1
  }
]
